<a href="https://colab.research.google.com/github/nibble1101/Payroll/blob/main/Payroll2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.colab import files

auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
import numpy as np
import datetime
from re import sub


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
worksheet = gc.open('sales-summary').sheet1
df = pd.DataFrame(worksheet.get_all_records())        # LOADING THE FILE INTO PANDA DATAFRAME


df.set_index("Sales", inplace = True)         # SETTING THE STARTING INDEX TO SELECT THE ROW 
tip = df.loc["Tip"]         # GETTING THE PANDA SERIES FOR THE TIP
date_list = []
tip_amount_list = []

for date,tip_amount in tip.items():         # CONVERTING THE DATES AND THE TIP AMOUNT TO CORRECT FORMAT, i.e. DATATIME & DOUBLE

  month, day, year = (int(x) for x in date.split('/'))    
  date_formatted = datetime.date(year, month, day)
  date_formatted = date_formatted.strftime("%m/%d/%y")
  date_list.append(date_formatted)

  tip_amount_formatted = round(float(sub(r'[^\d.]', '', tip_amount)),4)
  tip_amount_list.append(tip_amount_formatted)

tip.index = date_list         # CONVERTING THE INDEX OF THE PANDA SERIES

tip = pd.DataFrame(tip)         # CONVERTING THE PANDA SERIES TO DATAFRAME TO CHANGE THE ROW ENTRY OF TIPS.
tip["Tip"] = tip_amount_list          # UPDATINF THE TIP WITH NEW ENTRIES OF CORRECT FORMAT.

tip

,Tip
07/04/22,0.00
07/05/22,123.49
07/06/22,94.99
07/07/22,60.60
07/08/22,120.09
07/09/22,80.25
07/10/22,97.48
07/11/22,0.00
07/12/22,28.11
07/13/22,24.50


In [ ]:
worksheet = gc.open('service-charges').sheet1
df = pd.DataFrame(worksheet.get_all_records())        # LOADING THE FILE INTO PANDA DATAFRAME
df.set_index("Service Charge", inplace = True)         # SETTING THE STARTING INDEX TO SELECT THE ROW 
auto_gratuity = df.loc["Auto-Gratuity"]         # GETTING THE PANDA SERIES FOR THE AUTO GRATUITY
date_list = []
auto_gratuity_amount_list = []

for date,auto_gratuity_amount in auto_gratuity.items():         # CONVERTING THE DATES AND THE AUTO GRATUITY AMOUNT TO CORRECT FORMAT, i.e. DATATIME & DOUBLE

  month, day, year = (int(x) for x in date.split('/'))    
  date_formatted = datetime.date(year, month, day)
  date_formatted = date_formatted.strftime("%m/%d/%y")
  date_list.append(date_formatted)

  auto_gratuity_amount_formatted = round(float(sub(r'[^\d.]', '', auto_gratuity_amount)),4)
  auto_gratuity_amount_list.append(auto_gratuity_amount_formatted)

auto_gratuity.index = date_list         # CONVERTING THE INDEX OF THE PANDA SERIES

auto_gratuity = pd.DataFrame(auto_gratuity)         # CONVERTING THE PANDA SERIES TO DATAFRAME TO CHANGE THE ROW ENTRY OF TIPS.
auto_gratuity["Auto-Gratuity"] = auto_gratuity_amount_list          # UPDATINF THE AUTO GRATUITY WITH NEW ENTRIES OF CORRECT FORMAT.

auto_gratuity

,Auto-Gratuity
07/04/22,0.00
07/05/22,550.80
07/06/22,459.20
07/07/22,413.25
07/08/22,858.80
07/09/22,857.80
07/10/22,455.60
07/11/22,0.00
07/12/22,437.00
07/13/22,565.80


In [ ]:
worksheet = gc.open('points-table').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
point_table = pd.DataFrame.from_records(rows)

column_names = list(point_table.loc[0,:])
row_names = list(point_table[0])
column_names.pop(0)
row_names.pop(0)
point_table.drop(0, axis=0, inplace=True)
point_table.drop(0, axis=1, inplace=True)

point_table.columns = column_names
point_table.index = row_names

point_table_aux = point_table.apply(pd.to_numeric, errors='coerce').fillna(0.0)     # Converting all the values to double type

global_points_sum = sum(list(point_table_aux.loc["Points"]))

point_table

,Lead Server,Bartender,Server,Assistant Server,GM,Dishwasher,BOH,Host
Points,40,40,30,15,11,5,5,15
TipPool,Y,Y,Y,Y,N,Y,Y,Y
GratuityPool,Y,Y,Y,Y,Y,Y,Y,Y
Tip Percent,0.95,0.95,0.95,0.95,0.95,0.95,0.95,0.95
Gratuity Percent,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75
Processing Charge,0.95,0.95,0.95,0.95,0.95,0.95,0.95,0.95


In [ ]:
worksheet = gc.open('hours-export').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
df = pd.DataFrame.from_records(rows)

dates = list(df.iloc[:, 0])
df.drop(0, axis=1, inplace=True)
date_list = []
dates.pop(0)
dates.pop(0)

for date in dates:         # CONVERTING THE DATES AND THE AUTO GRATUITY AMOUNT TO CORRECT FORMAT, i.e. DATATIME & DOUBLE

  month, day, year = (int(x) for x in date.split('/'))    
  date_formatted = datetime.date(year, month, day)
  date_formatted = date_formatted.strftime("%m/%d/%Y")
  date_list.append(date_formatted)

date_list.append("Grand Total")


# Setting up the column names by name to access them.
positions = list(df.iloc[0]) # Getting the position.
names = list(df.iloc[1]) # Getting the names.
column_labels = []

# Creating the column name in the format "Name - Position"
for i in range(0, len(names)):
  column_labels.append(names[i] + " - *" + positions[i] + "*")
df.columns = column_labels

# # Removing the "Position - Row Labels"
# column_labels.pop(0)

df.drop([0,1], axis=0, inplace=True) # Removing the positions and the row labels rows from the dataframe

# Converting all the possible numeric values to numeric
df = df.apply(pd.to_numeric, errors='coerce').fillna(df)
df.reset_index(drop=True, inplace=True)

# Summation
df["Grand Total"] = df[column_labels].apply(pd.to_numeric, errors='coerce').fillna(0.0).sum(axis=1) # Summing each row
column_labels.append("Grand Total")
sum_column = list(df[column_labels].apply(pd.to_numeric, errors='coerce').fillna(0.0).sum(axis=0)) # Summing each column

df.loc[len(df.index)] = sum_column

# Updating the index names for the rows.
df.index = date_list

hour_sheet = df
hour_sheet = hour_sheet.apply(pd.to_numeric, errors='coerce').fillna(0.0)

global_employee_position = list(df.columns)
global_employee_position.pop()    # Removing 'Grand Total' From the list

global_dates = list(df.index)
global_dates.pop()    # Removing "Grand Total" from the list

global_total_hours_worked_per_week = sum_column

hour_sheet

,Alexander - *Server*,Nadia - *Assistant Server*,Autumn - *Server*,Ian - *Server*,Darren - *GM*,Dorothy - *Lead Server*,Edwin - *Dishwasher*,Grace - *Assistant Server*,Jeremy - *Lead Server*,Mary - *Lead Server*,Rachel - *Lead Server*,Sydney - *Server*,Deepak - *Server*,BOH - *BOH*,Grand Total
07/05/22,6.13,0.00,4.00,0.00,8.0,0.00,6.95,3.62,4.52,0.00,0.00,0.00,6.08,32.0,71.30
07/06/22,4.38,0.00,0.00,0.00,8.0,0.00,6.40,0.00,5.37,5.42,6.53,0.00,0.00,32.0,68.10
07/07/22,0.00,0.00,0.00,0.00,8.0,0.00,7.05,0.00,0.00,0.00,6.30,0.00,6.75,32.0,60.10
07/08/22,4.85,3.87,0.00,0.00,8.0,3.80,8.05,3.80,4.67,4.63,4.78,5.53,6.53,36.0,94.51
07/09/22,5.33,3.45,0.00,0.00,8.0,0.00,7.82,3.12,5.10,5.83,4.73,3.75,6.18,36.0,89.31
07/10/22,0.00,3.43,0.00,0.00,0.0,7.58,6.60,3.43,0.00,5.57,0.00,5.95,5.78,32.0,70.34
07/12/22,0.00,0.00,4.00,0.00,8.0,0.00,7.18,0.00,0.00,0.00,6.78,0.00,7.23,32.0,65.19
07/13/22,0.00,0.00,0.00,0.00,8.0,0.00,6.67,2.32,0.00,3.77,6.30,2.93,5.80,32.0,67.79
07/14/22,0.00,0.00,0.00,0.00,8.0,0.00,6.12,0.00,0.00,5.90,6.48,0.00,0.00,32.0,58.50
07/15/22,0.00,3.52,4.27,3.83,8.0,3.47,7.43,3.57,0.00,4.90,3.92,4.77,6.25,36.0,89.93


In [ ]:
# CALCULATING GRATUITY FOR EACH EMPLOYEE
calculated_auto_gratuity = pd.DataFrame(columns = global_employee_position, index = global_dates)

positions = list(point_table.columns)
hrs_per_day_per_position = {}
for position in positions:
  hrs_per_day_per_position[position] = 0.0


for date in global_dates:

  total_point_hours = 0.0
  gratuity_on_date = auto_gratuity.loc[date,"Auto-Gratuity"]
  # print(gratuity_on_date)

  for employee in global_employee_position:

    position = employee.partition("*")[2].partition("*")[0]   # Extracting the position from the string.

    if point_table.loc["GratuityPool", position] == 'Y':
      total_point_hours+= float(hour_sheet.loc[date,employee]) * float(point_table.loc["Points",position])
      # print(float(hour_sheet.loc[date,employee]) , float(point_table.loc["Points",position]), position)

  # print(date,total_point_hours)

  for employee in global_employee_position:

    position = employee.partition("*")[2].partition("*")[0]   # Extracting the position from the string.

    if point_table.loc["GratuityPool", position] == 'Y':
      hrs_worked = float(hour_sheet.loc[date,employee])
      employee_points = float(point_table.loc["Points",position])
      gratuity_percent = point_table_aux.loc["Gratuity Percent", position]
      processing_charge = point_table_aux.loc["Processing Charge", position]
      gratuity_share = (hrs_worked * employee_points * gratuity_on_date * gratuity_percent * processing_charge) / total_point_hours


      calculated_auto_gratuity.loc[date, employee]  = gratuity_share

    else:
      calculated_auto_gratuity.loc[date, employee]  = 0.0

# calculated_auto_gratuity


# Summation:
row_labels = global_dates[:]
column_labels = global_employee_position[:]
calculated_auto_gratuity["Grand Total"] = calculated_auto_gratuity[column_labels].apply(pd.to_numeric, errors='coerce').fillna(0.0).sum(axis=1) # Summing each row
column_labels.append("Grand Total")
row_labels.append("Grand Total")
sum_column = list(calculated_auto_gratuity[column_labels].apply(pd.to_numeric, errors='coerce').fillna(0.0).sum(axis=0)) # Summing each column
calculated_auto_gratuity.loc[len(calculated_auto_gratuity.index)] = sum_column

calculated_auto_gratuity.index = row_labels

calculated_auto_gratuity

,Alexander - *Server*,Nadia - *Assistant Server*,Autumn - *Server*,Ian - *Server*,Darren - *GM*,Dorothy - *Lead Server*,Edwin - *Dishwasher*,Grace - *Assistant Server*,Jeremy - *Lead Server*,Mary - *Lead Server*,Rachel - *Lead Server*,Sydney - *Server*,Deepak - *Server*,BOH - *BOH*,Grand Total
07/05/22,71.872365,0.0,46.89877,0.0,34.392431,0.0,13.581102,21.221693,70.660814,0.0,0.0,0.0,71.286131,62.531693,392.445000
07/06/22,38.934479,0.0,0.0,0.0,26.074842,0.0,9.481761,0.0,63.646318,64.238928,77.39487,0.0,0.0,47.408803,327.180000
07/07/22,0.0,0.0,0.0,0.0,35.121349,0.0,14.068495,0.0,0.0,0.0,100.574771,0.0,80.819013,63.856998,294.440625
07/08/22,54.095712,21.582516,0.0,0.0,32.717681,56.512359,14.964621,21.192135,69.450715,68.855848,71.086599,61.680265,72.83402,66.92253,611.895000
07/09/22,65.59592,21.229449,0.0,0.0,36.100319,0.0,16.040028,19.198806,83.687103,95.665845,77.615685,46.150976,76.056808,73.841561,611.182500
07/10/22,0.0,14.228524,0.0,0.0,0.0,83.850118,9.126167,14.228524,0.0,61.615456,0.0,49.364268,47.95386,44.248083,324.615000
07/12/22,0.0,0.0,41.887332,0.0,30.717377,0.0,12.531293,0.0,0.0,0.0,94.66537,0.0,75.711352,55.849776,311.362500
07/13/22,0.0,0.0,0.0,0.0,36.168283,0.0,13.706957,14.302912,0.0,61.979284,103.572809,36.127182,71.514559,65.760514,403.132500
07/14/22,0.0,0.0,0.0,0.0,18.393513,0.0,6.395926,0.0,0.0,49.328056,54.177255,0.0,0.0,33.44275,161.737500
07/15/22,0.0,20.288866,49.223555,44.151338,33.814776,53.335124,14.275215,20.57706,0.0,75.314729,60.251783,54.987437,72.048529,69.166588,567.435000


In [ ]:
# CALCULATING TIP FOR EACH EMPLOYEE

calculated_tip = pd.DataFrame(columns = global_employee_position, index = global_dates)

positions = list(point_table.columns)
hrs_per_day_per_position = {}
for position in positions:
  hrs_per_day_per_position[position] = 0.0


for date in global_dates:

  total_point_hours = 0.0
  tip_on_date = tip.loc[date,"Tip"]

  for employee in global_employee_position:

    position = employee.partition("*")[2].partition("*")[0]   # Extracting the position from the string.

    if point_table.loc["TipPool", position] == 'Y':
      total_point_hours+= float(hour_sheet.loc[date,employee]) * float(point_table.loc["Points",position])
      # print(float(hour_sheet.loc[date,employee]) , float(point_table.loc["Points",position]), position)

  # print(date,total_point_hours)

  for employee in global_employee_position:

    position = employee.partition("*")[2].partition("*")[0]   # Extracting the position from the string.

    if point_table.loc["TipPool", position] == 'Y':
      hrs_worked = float(hour_sheet.loc[date,employee])
      employee_points = float(point_table.loc["Points",position])
      tip_percent = point_table_aux.loc["Tip Percent", position]
      processing_charge = point_table_aux.loc["Processing Charge", position]
      tip_share = (hrs_worked * employee_points * tip_on_date * tip_percent * processing_charge) / total_point_hours


      calculated_tip.loc[date, employee]  = tip_share
    
    else:
      calculated_tip.loc[date, employee]  = 0.0

calculated_tip


# Summation:
row_labels = global_dates[:]
column_labels = global_employee_position[:]
calculated_tip["Grand Total"] = calculated_tip[column_labels].apply(pd.to_numeric, errors='coerce').fillna(0.0).sum(axis=1) # Summing each row
column_labels.append("Grand Total")
row_labels.append("Grand Total")
sum_column = list(calculated_tip[column_labels].apply(pd.to_numeric, errors='coerce').fillna(0.0).sum(axis=0)) # Summing each column
calculated_tip.loc[len(calculated_tip.index)] = sum_column

calculated_tip.index = row_labels

calculated_tip

,Alexander - *Server*,Nadia - *Assistant Server*,Autumn - *Server*,Ian - *Server*,Darren - *GM*,Dorothy - *Lead Server*,Edwin - *Dishwasher*,Grace - *Assistant Server*,Jeremy - *Lead Server*,Mary - *Lead Server*,Rachel - *Lead Server*,Sydney - *Server*,Deepak - *Server*,BOH - *BOH*,Grand Total
07/05/22,22.371451,0.0,14.59801,0.0,0.0,0.0,4.22734,6.6056,21.994335,0.0,0.0,0.0,22.188975,19.464014,111.449725
07/06/22,11.085142,0.0,0.0,0.0,0.0,0.0,2.699578,0.0,18.120918,18.289641,22.035306,0.0,0.0,13.49789,85.728475
07/07/22,0.0,0.0,0.0,0.0,0.0,0.0,2.967103,0.0,0.0,0.0,21.211632,0.0,17.045062,13.467703,54.691500
07/08/22,10.122909,4.038728,0.0,0.0,0.0,10.575136,2.800324,3.965676,12.996285,12.884968,13.302407,11.542204,13.629402,12.523187,108.381225
07/09/22,8.261125,2.673628,0.0,0.0,0.0,0.0,2.020075,2.41789,10.539521,12.048119,9.774889,5.812236,9.578565,9.299577,72.425625
07/10/22,0.0,3.856151,0.0,0.0,0.0,22.724682,2.473333,3.856151,0.0,16.698744,0.0,13.378482,12.99624,11.991917,87.975700
07/12/22,0.0,0.0,3.786459,0.0,0.0,0.0,1.132782,0.0,0.0,0.0,8.557397,0.0,6.844025,5.048612,25.369275
07/13/22,0.0,0.0,0.0,0.0,0.0,0.0,0.825906,0.861815,0.0,3.734532,6.240729,2.176826,4.309075,3.962368,22.111250
07/14/22,0.0,0.0,0.0,0.0,0.0,0.0,5.786659,0.0,0.0,44.629138,49.016409,0.0,0.0,30.257043,129.689250
07/15/22,0.0,3.220722,7.813911,7.00873,0.0,8.466595,2.266095,3.266471,0.0,11.955711,9.564569,8.728889,11.437223,10.979734,84.708650


In [ ]:
import pandas
# CALCULATING THE SUM OF TIPS AND GRATUITY

# Adding the total hours row in the gratuity tip sum.

gratuity_tip_sum = calculated_auto_gratuity.copy()
gratuity_tip_sum = gratuity_tip_sum.add(calculated_tip, fill_value=0)

gratuity_tip_sum

,Alexander - *Server*,Nadia - *Assistant Server*,Autumn - *Server*,Ian - *Server*,Darren - *GM*,Dorothy - *Lead Server*,Edwin - *Dishwasher*,Grace - *Assistant Server*,Jeremy - *Lead Server*,Mary - *Lead Server*,Rachel - *Lead Server*,Sydney - *Server*,Deepak - *Server*,BOH - *BOH*,Grand Total
07/05/22,94.243816,0.0,61.49678,0.0,34.392431,0.0,17.808443,27.827293,92.655149,0.0,0.0,0.0,93.475106,81.995707,503.894725
07/06/22,50.019622,0.0,0.0,0.0,26.074842,0.0,12.181339,0.0,81.767235,82.528569,99.430176,0.0,0.0,60.906693,412.908475
07/07/22,0.0,0.0,0.0,0.0,35.121349,0.0,17.035598,0.0,0.0,0.0,121.786403,0.0,97.864074,77.324701,349.132125
07/08/22,64.218621,25.621244,0.0,0.0,32.717681,67.087494,17.764945,25.15781,82.447,81.740815,84.389006,73.222469,86.463422,79.445717,720.276225
07/09/22,73.857045,23.903077,0.0,0.0,36.100319,0.0,18.060103,21.616696,94.226624,107.713964,87.390575,51.963212,85.635373,83.141139,683.608125
07/10/22,0.0,18.084675,0.0,0.0,0.0,106.5748,11.5995,18.084675,0.0,78.3142,0.0,62.74275,60.9501,56.24,412.590700
07/12/22,0.0,0.0,45.673791,0.0,30.717377,0.0,13.664076,0.0,0.0,0.0,103.222767,0.0,82.555377,60.898388,336.731775
07/13/22,0.0,0.0,0.0,0.0,36.168283,0.0,14.532863,15.164727,0.0,65.713816,109.813538,38.304008,75.823634,69.722882,425.243750
07/14/22,0.0,0.0,0.0,0.0,18.393513,0.0,12.182585,0.0,0.0,93.957195,103.193665,0.0,0.0,63.699793,291.426750
07/15/22,0.0,23.509588,57.037466,51.160069,33.814776,61.801719,16.54131,23.843531,0.0,87.270439,69.816352,63.716326,83.485752,80.146322,652.143650


In [ ]:
per_hour_gratuity_tip = gratuity_tip_sum.copy()
per_hour_gratuity_tip = per_hour_gratuity_tip.div(hour_sheet.values).fillna(0.0)




per_hour_gratuity_tip

,Alexander - *Server*,Nadia - *Assistant Server*,Autumn - *Server*,Ian - *Server*,Darren - *GM*,Dorothy - *Lead Server*,Edwin - *Dishwasher*,Grace - *Assistant Server*,Jeremy - *Lead Server*,Mary - *Lead Server*,Rachel - *Lead Server*,Sydney - *Server*,Deepak - *Server*,BOH - *BOH*,Grand Total
07/05/22,15.374195,0.000000,15.374195,0.000000,4.299054,0.000000,2.562366,7.687098,20.498927,0.000000,0.000000,0.000000,15.374195,2.562366,7.067247
07/06/22,11.420005,0.000000,0.000000,0.000000,3.259355,0.000000,1.903334,0.000000,15.226673,15.226673,15.226673,0.000000,0.000000,1.903334,6.063267
07/07/22,0.000000,0.000000,0.000000,0.000000,4.390169,0.000000,2.416397,0.000000,0.000000,0.000000,19.331175,0.000000,14.498381,2.416397,5.809187
07/08/22,13.240953,6.620476,0.000000,0.000000,4.089710,17.654604,2.206825,6.620476,17.654604,17.654604,17.654604,13.240953,13.240953,2.206825,7.621164
07/09/22,13.856856,6.928428,0.000000,0.000000,4.512540,0.000000,2.309476,6.928428,18.475809,18.475809,18.475809,13.856856,13.856856,2.309476,7.654329
07/10/22,0.000000,5.272500,0.000000,0.000000,0.000000,14.060000,1.757500,5.272500,0.000000,14.060000,0.000000,10.545000,10.545000,1.757500,5.865662
07/12/22,0.000000,0.000000,11.418448,0.000000,3.839672,0.000000,1.903075,0.000000,0.000000,0.000000,15.224597,0.000000,11.418448,1.903075,5.165390
07/13/22,0.000000,0.000000,0.000000,0.000000,4.521035,0.000000,2.178840,6.536520,0.000000,17.430720,17.430720,13.073040,13.073040,2.178840,6.272957
07/14/22,0.000000,0.000000,0.000000,0.000000,2.299189,0.000000,1.990619,0.000000,0.000000,15.924948,15.924948,0.000000,0.000000,1.990619,4.981654
07/15/22,0.000000,6.678860,13.357720,13.357720,4.226847,17.810294,2.226287,6.678860,0.000000,17.810294,17.810294,13.357720,13.357720,2.226287,7.251681


In [ ]:
# SAVING THE FILES AND DOWNLOADING THEM

# ADDING TOTAL HOURS WORKED

gratuity_tip_sum.loc[len(gratuity_tip_sum.index)] = global_total_hours_worked_per_week

gratuity_tip_sum.rename({gratuity_tip_sum.index[-1]: 'Total hours worked.'}, inplace=True)

# GETTING PER HOUR GRATUITY TIP SUM:

global_total_per_hour_gratuity_tip_sum = per_hour_gratuity_tip.loc["Grand Total"]
gratuity_tip_sum.loc[len(gratuity_tip_sum.index)] = global_total_per_hour_gratuity_tip_sum
gratuity_tip_sum.rename({gratuity_tip_sum.index[-1]: 'Per hour gratuity tip sum.'}, inplace=True)

with pd.ExcelWriter('Calculation.xlsx') as writer:
  calculated_auto_gratuity.to_excel(writer, sheet_name='Auto-Gratuity')
  calculated_tip.to_excel(writer, sheet_name='Tip')
  gratuity_tip_sum.to_excel(writer, sheet_name='Gratuity tip sum')
  per_hour_gratuity_tip.to_excel(writer, 'Per hour stats')

files.download('Calculation.xlsx')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>